In [ ]:
import gymnasium as gym
import numpy as np
import pandas
import seaborn as sns
import torch
from einops import einsum, pack, rearrange, repeat
from matplotlib import pyplot as plt
from torch import nn

from legato.nets import Perceptron, TransitionModel
from legato.sampler import PBallSampler

In [ ]:
np_rng = np.random.default_rng(0)

In [ ]:
# Load data from data.npz
data = np.load("../data.npz")
indices = np.load("../trained_net_params/indices.npz")
train_indices, test_indices = indices["train_indices"], indices["test_indices"]

observations_train = data["observations"][train_indices]
actions_train = data["actions"][train_indices]

observations_test = torch.tensor(data["observations"][test_indices], dtype=torch.float32, device="cuda")
actions_test = torch.tensor(data["actions"][test_indices], dtype=torch.float32, device="cuda")

In [ ]:
# Load models
state_encoder = torch.load("../trained_net_params/state_encoder.pt")
action_encoder = torch.load("../trained_net_params/action_encoder.pt")
transition_model = torch.load("../trained_net_params/transition_model.pt")
state_decoder = torch.load("../trained_net_params/state_decoder.pt")
action_decoder = torch.load("../trained_net_params/action_decoder.pt")

In [ ]:
flat_states = rearrange(observations_test, "... f -> (...) f")
flat_actions = rearrange(actions_test, "... f -> (...) f")

# SHuffle
flat_states = flat_states[np_rng.permutation(flat_states.shape[0])]
flat_actions = flat_actions[np_rng.permutation(flat_actions.shape[0])]

flat_states = flat_states[:2048]
flat_actions = flat_actions[:2048]

In [ ]:
# Get stdev of encoded states and actions across each element
with torch.no_grad():
    encoded_states = state_encoder(flat_states)
    state_std = torch.std(encoded_states, dim=0)

    encoded_actions = action_encoder((flat_actions, flat_states))

    recovered_states = state_decoder(encoded_states)
    recovered_actions = action_decoder((encoded_actions, encoded_states))

    action_std = torch.std(encoded_actions, dim=0)

In [ ]:
encoded_states

In [ ]:
encoded_actions

In [ ]:
print(f"State std: {state_std.cpu().numpy()}\nAction std: {action_std.cpu().numpy()}")

In [ ]:
transition_batch_size = 32
# Test forward model
traj_inds = torch.randint(
    0, observations_test.shape[0], (transition_batch_size,), device="cuda"
)
test_start_inds = torch.randint(
    0, int(observations_test.shape[-2] // 1.1), (transition_batch_size,), device="cuda"
)

test_states = observations_test[traj_inds]
test_actions = actions_test[traj_inds]

start_states = observations_test[traj_inds, test_start_inds]

In [ ]:
start_states.shape

In [ ]:
latent_start_states = state_encoder(start_states)

latent_traj_actions = action_encoder((test_actions, test_states))

latent_pred_fut_states = transition_model(
    latent_start_states, latent_traj_actions, start_indices=test_start_inds
)

In [ ]:
latent_fut_states = state_encoder(test_states)

traj_ind = traj_inds[0]
start_ind = test_start_inds[0]

latent_fut_states_select = latent_fut_states[0, start_ind:]
latent_pred_fut_states_select = latent_pred_fut_states[0, start_ind:]

In [ ]:
torch.stack([latent_fut_states_select, latent_pred_fut_states_select], dim=-1)

In [ ]:
# Measure error per time into the future

time_into_future = torch.arange(
    latent_fut_states.shape[-2], device="cuda"
)[None] - test_start_inds[..., None]

In [ ]:
mae_errors = torch.mean(
    torch.abs(latent_fut_states - latent_pred_fut_states), dim=-1
)

In [ ]:
mae_errors_flat = rearrange(mae_errors, "... -> (...)").detach().cpu().numpy()
time_into_future_flat = (
    rearrange(time_into_future, "... -> (...)").detach().cpu().numpy()
)

df = pandas.DataFrame.from_dict(
    {"mae_error": mae_errors_flat, "time_into_future": time_into_future_flat},
)
df = df[df["time_into_future"] >= 0]

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(data=df, x="time_into_future", y="mae_error", ax=ax)
ax.set_xlim(0, 1024)

In [ ]:
latent_state_sampler = PBallSampler(
    4, 1, 2.0, device="cuda"
)

In [ ]:
latent_states = state_encoder(flat_states[:512])

In [ ]:
class CoverageLoss(nn.Module):

    def __init__(
        self,
        latent_sampler,
        norm_p=1,
        latent_samples=2048,
        selection_tail_size=4,
        far_sample_count=64,
        pushing_sample_size=4,
        loss_function=nn.L1Loss(),  # nn.HuberLoss(),
    ):
        super().__init__()

        self.latent_sampler = latent_sampler
        self.norm_p = norm_p
        self.latent_samples = latent_samples
        self.selection_tail_size = selection_tail_size
        self.far_sample_count = far_sample_count
        self.pushing_sample_size = pushing_sample_size
        self.loss_function = loss_function

    def forward(self, latents):
        # penalize for empty space within the state space
        # Sample random points in the latent space

        proposals = self.latent_sampler(self.latent_samples)
        points = torch.cat([proposals, latents], dim=0)
        distances = torch.cdist(proposals, points, p=self.norm_p)

        in_point_mask = torch.zeros(
            len(points), dtype=torch.bool, device=latents.device
        )
        in_point_mask[self.latent_samples :] = True
        in_point_mask[self.latent_samples :] = True
        far_samples = torch.zeros(
            self.far_sample_count,
            latents.shape[-1],
            dtype=latents.dtype,
            device=latents.device,
        )
        for i in range(self.far_sample_count):
            in_point_dists = distances[:, in_point_mask]
            tail_distances = torch.topk(
                in_point_dists, self.selection_tail_size, dim=-1, largest=False
            ).values.mean(-1)
            # Mask out already selected points
            tail_distances[in_point_mask[: self.latent_samples]] = -torch.inf
            farthest_index = torch.topk(tail_distances, 1, dim=-1).indices
            in_point_mask[farthest_index] = True
            far_samples[i] = proposals[farthest_index]

        # Now make the states by the latent states closer to the farthest samples
        empty_space_dists = torch.cdist(far_samples, latents, p=self.norm_p)
        close_empty_space_dists = torch.topk(
            empty_space_dists, self.pushing_sample_size, dim=-1, largest=False
        ).values.mean(-1)
        space_coverage_loss = self.loss_function(
            close_empty_space_dists, torch.zeros_like(close_empty_space_dists)
        ).mean()

        return far_samples

In [ ]:
scl = CoverageLoss(latent_state_sampler, norm_p=1, latent_samples=16_384, selection_tail_size=4, far_sample_count=64, pushing_sample_size=4)

In [ ]:
far_samples = scl(latent_states)
sdf = pandas.DataFrame.from_records(
    np.concatenate(
        [
            np.concatenate(
                [
                    latent_states.detach().cpu().numpy(),
                    np.zeros((len(latent_states), 1)),
                ],
                axis=1,
            ),
            np.concatenate(
                [
                    far_samples.detach().cpu().numpy(),
                    np.ones((len(far_samples), 1)),
                ],
                axis=1,
            ),
        ],
    ),
    columns=["e0", "e1", "e2", "e3", "samp"],
)

In [ ]:
sns.pairplot(sdf, hue="samp", palette="icefire")

In [ ]:
latent_action_sampler = PBallSampler(
    2, 1, 1.0, device="cuda"
)

In [ ]:
latent_actions = action_encoder((flat_actions[:512], flat_states[:512]))

In [ ]:
acl = CoverageLoss(latent_action_sampler, norm_p=1, latent_samples=16_384, selection_tail_size=4, far_sample_count=64, pushing_sample_size=4)

In [ ]:
far_samples = acl(latent_actions)
adf = pandas.DataFrame.from_records(
    np.concatenate(
        [
            np.concatenate(
                [
                    latent_actions.detach().cpu().numpy(),
                    np.zeros((len(latent_actions), 1)),
                ],
                axis=1,
            ),
            np.concatenate(
                [
                    far_samples.detach().cpu().numpy(),
                    np.ones((len(far_samples), 1)),
                ],
                axis=1,
            ),
        ],
    ),
    columns=["e0", "e1", "samp"],
)

In [ ]:
sns.pairplot(adf, hue="samp", palette="icefire")